In [14]:
# Definitions and functions
%matplotlib inline
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
import yaml
from scipy.io import wavfile
from scipy.signal import spectrogram
import logging
from IPython.display import display
matplotlib.style.use('ggplot')


# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
print 'Computer: ' + comp_name

if 'txori' in comp_name or 'passaro' in comp_name or 'lintu' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'analysis-tools'))
sys.path.append(os.path.join(repos_folder, 'swissknife'))

import soundtools as st
import Kwik as oe
from basic_viewing import h5_functions as h5

from file_tools import experiment as et
from h5tools import tables as h5t

Computer: passaro


In [15]:
def append_rows(table, new_data):
    rows = table.shape[0]
    more_rows = new_data.shape[0]
    table.resize(rows + more_rows, axis=0)
    if table.size == (rows + more_rows):
        table[rows:] = new_data
    else:
        table[rows:,:] = new_data

def unlimited_rows_data(group, table_name, data):
    try:
        table = group.create_dataset(table_name, 
                                     data=data, 
                                     dtype=data.dtype, 
                                     maxshape={None, None})
    except RuntimeError as e:
        if 'Name already exists' in str(e):
            table = group[table_name]
            append_rows(table, data)
        else:
            raise
    return table

def store_motiff(ev_file, rec, bout_starts, motiff_name):
    motiff_group = ev_file.require_group('/event_types/singing/{0}'.format(motiff_name))
    t_table = unlimited_rows_data(motiff_group, 'time_samples', bout_starts)
    r_table = unlimited_rows_data(motiff_group, 'recording', 
                                  data=np.ones_like(bout_starts)*int(rec))

In [16]:
bird_id = 'z009'
sess_id = 'anesth_surface_2016-11-04_16-19-47_5'
super_sess_id = 'day-anesth_surface_2016-11-04_5'

fn = et.file_names(bird_id, sess_id)

ev_file_path = os.path.join(fn['folders']['rw'], 'experiment1.kwe')
kwd_file_path = os.path.join(fn['folders']['rw'], 'experiment1_100.raw.kwd')

new_fn = et.file_names(bird_id, sess_id)
new_ev_file_path =  os.path.join(new_fn['folders']['ss'], new_fn['files']['stm'])

h5file = h5py.File(ev_file_path, 'r')

In [17]:
def get_messages(h5file, rec_id=None, node=None):
    rec_table = h5file['event_types']['Messages']['events']['recording'][:]
    t_table = h5file['event_types']['Messages']['events']['time_samples'][:]
    text_table = h5file['event_types']['Messages']['events']['user_data']['Text'][:]
    node_table = h5file['event_types']['Messages']['events']['user_data']['nodeID'][:]
    
    return pd.DataFrame.from_items([('t', t_table),
                                    ('text', text_table),
                                    ('rec', rec_table),
                                    ('node', node_table)])

def get_trial_times(kwe_file):
    all_msg = get_messages(kwe_file)
    trials_df = all_msg[all_msg['text'].str.contains('trial_number')]
    trial_times = trials_df.t.values.astype(np.int)
    trial_numbers = [int(x.split()[-1]) for x in trials_df.text.values]
    return pd.DataFrame.from_items([('t', trial_times - get_start_rec_times(kwe_file)), 
                                    ('trial', trial_numbers), 
                                    ('rec', trials_df['rec'].values)])

def get_wave_times(kwe_file, wave_file=None):
    all_msg = get_messages(kwe_file)
    wav_df = all_msg[all_msg['text'].str.contains('play_wav')]
    wav_times = wav_df.t.values.astype(np.int)
    wav_names = [(x.split()[-1].split('/')[-1].split('.')[0]) for x in wav_df.text.values]
    
    wav_table =  pd.DataFrame.from_items([('t', wav_times - get_start_rec_times(kwe_file)), 
                                          ('stim_name', wav_names), 
                                          ('rec', wav_df['rec'].values)])
    
    if wave_file is not None:
        wav_table = wav_table[wav_table['stim_name']==wave_file]
    return wav_table

def get_trial_bounds(stamp, tr_df):
    trial_n = max(tr_df[tr_df.t<stamp].trial)
    t_start = tr_df[tr_df.trial==trial_n].t.values[0]
    try:
        t_end = tr_df[tr_df.trial==trial_n + 1].t.values[0]
    except:
        t_end = None
    return [t_start, t_end]
    
def get_start_rec_times(kwe_file):
    all_msg = get_messages(kwe_file)
    start_rec_df = all_msg[all_msg['text'].str.contains('start time')]
    start_times = [long(x.split()[-1].split('@')[0]) for x in start_rec_df.text.values]
    assert(len(start_times)==1), "more or less than one recording start time"
    return start_times[0]

def get_stims_list(kwe_file):
    stims_table = get_wave_times(kwe_file)
    return np.unique(stims_table.stim_name)
    
def find_first_peak(x, thresh_factor=0.5):
    x = x - np.mean(x)
    thresh = np.max(x) * thresh_factor
    # find the peaks naively
    a = x[1:-1] - x[2:]
    b = x[1:-1] - x[:-2]
    c = x[1:-1]
    max_pos = np.where((a > 0) & (b > 0) & (c > thresh))[0]+1
    return max_pos[0]

def find_wav_onset(dset, chan, stamp, tr_df):
    [start, end] = get_trial_bounds(stamp, tr_df)
    trial_frame = h5.load_table_slice(dset, np.arange(start, end), [chan])
    onset_in_trial = find_first_peak(trial_frame)
    return start + onset_in_trial

def get_stim_starts(kwe_file, kwd_file, rec, tag_chan, stim_name):
    data_set = h5.get_data_set(kwd_file, rec)
    trials = get_trial_times(kwe_file)
    stim_times = get_wave_times(kwe_file, stim_name)
    precise_starts = np.array([find_wav_onset(data_set, tag_chan, stamp, trials) for stamp in stim_times.t.values])
    return precise_starts


In [18]:
# get all the song presentation events for a supersession
bird_id = 'z009'
sess_id = 'anesth_surface_2016-11-04_16-19-47_5'
super_sess_id = 'day-anesth_surface_2016-11-04_5'

ss_fn = et.file_names(bird_id, super_sess_id)
mot_file_path = et.file_path(ss_fn, 'ss', 'sng')
super_sess_path = et.file_path(ss_fn, 'ss', 'ss_raw')

rec_list = et.get_rec_list(bird_id, super_sess_id)
for rec in rec_list:
    #get the rec events file
    rec_origin = h5.get_rec_origin(super_sess_path, rec)
    print rec_origin
    origin_fn = et.file_names(bird_id, rec_origin['sess'], base=rec_origin['base'])
    
    rec_ev_file_path = et.file_path(origin_fn, 'rw', 'evt')
    rec_kwd_file_path = et.file_path(origin_fn, 'rw', 'ss_raw')
    
    # read the raw parameters file and get the tag channel
    par_file_path = et.file_path(et.file_names(bird_id, rec_origin['sess']), 'rw', 'par')
    with open(os.path.join(fn['folders']['rw'], fn['files']['par']), 'r') as f:
        pars = yaml.load(f)
    tag_chan = int(pars['channel_config']['sts'])
    
    with h5py.File(rec_ev_file_path, 'r') as rec_ev_file:
        for stim_id in get_stims_list(rec_ev_file):
            print stim_id
            store_starts = get_stim_starts(rec_ev_file, rec_kwd_file_path, 
                                           rec_origin['rec'], tag_chan, stim_id)
            print store_starts
            with h5py.File(mot_file_path, 'a') as mot_file:
                store_motiff(mot_file, rec, store_starts, stim_id)

{'rec': 0, 'sess': 'anesth_surface_2016-11-04_16-19-47_5', 'base': 'experiment1_100', 'bird': 'z009'}
bos_rev_tag
[ 3731893  5524029  7871678 10700714 11763247 12551828 14698095 16674598
 17440619 19539098 21346349 21891631 22390158 24287525 24941381 24941381
 35558393 38395349 42023325 45124006 47142178]
bos_tag
[  181627  1361213  1736040  4332230  6175688  9464440 10973913 11235900
 13268113 24663894 26360329 27848392 28028205 28897684 30077966 31329450
 33361182 33766116 36269309 39545844 41191752 43022980 44801491 45801113
 47652727]


In [19]:
os.path.join(fn['folders']['rw'], fn['files']['par'])

'/mnt/cube/earneodo/bci_zf/raw_data/z009/anesth_surface_2016-11-04_16-19-47_5/experiment.par.yml'

In [21]:
store_starts

array([  181627,  1361213,  1736040,  4332230,  6175688,  9464440,
       10973913, 11235900, 13268113, 24663894, 26360329, 27848392,
       28028205, 28897684, 30077966, 31329450, 33361182, 33766116,
       36269309, 39545844, 41191752, 43022980, 44801491, 45801113, 47652727])

In [12]:
pars = et.get_parameters(bird_id, rec_origin['sess'], location='rw')

In [20]:
pars.

{'channel_config': {'mic': 67,
  'neural': [15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46],
  'ser': 69,
  'sts': 68},
 'probe': {'coord': [2300, 0, 0], 'model': 'None', 'serial': 'lhs2001b2'},
 'rec_config': {'processors': {'data': 100, 'evt': 102}},
 'search_motiff': {'corr_thresh': 0.85,
  'filt_hi': 400,
  'filt_lo': 10000,
  'motiff_patterns': {'bos_rev_tag': 'bos_rev_tag.wav',
   'bos_tag': 'bos_tag.wav'},
  'n_overlap': 192,
  'n_window': 256,
  'onset_resolution': 1,
  'rms_threshold': 2.0,
  'rolling_method': 'pandas',
  'sigma_factor': 0.001,
  'window_env': 500}}

In [7]:
os.path.join(fn['folders']['rw'], fn['files']['par'])

'/mnt/cube/earneodo/bci_zf/raw_data/z009/anesth_surface_2016-11-04_16-19-47_5/experiment.par.yml'

In [6]:
fn

{'files': {'base': 'experiment',
  'cand': 'experiment.mot.h5',
  'evt': 'experiment.kwe',
  'kk_par': 'params.prm',
  'kk_prb': '*.prb',
  'ks_cfg': 'config.m',
  'ks_map': 'chanMap.mat',
  'ks_mas': 'master.m',
  'ks_par': 'params.py',
  'mic': 'experiment-rec_000.mic.wav',
  'par': 'experiment.par.yml',
  'sng': 'experiment.sng.kwe',
  'ss_bin': 'experiment.dat',
  'ss_par': 'experiment.par.yml',
  'ss_raw': 'experiment.raw.kwd',
  'stm': 'experiment.stm.kwe',
  'sts': 'experiment-rec_000.sts.wav'},
 'folders': {'kai': '/mnt/cube/kai/results/z009/anesth_surface_2016-11-04_16-19-47_5',
  'prb': '/mnt/cube/earneodo/bci_zf/probes',
  'raw': '/usr/local/experiment/raw_data/z009/anesth_surface_2016-11-04_16-19-47_5',
  'rw': '/mnt/cube/earneodo/bci_zf/raw_data/z009/anesth_surface_2016-11-04_16-19-47_5',
  'ss': '/mnt/cube/earneodo/bci_zf/ss_data/z009/anesth_surface_2016-11-04_16-19-47_5',
  'stim': '/mnt/cube/earneodo/bci_zf/stim_data/z009/anesth_surface_2016-11-04_16-19-47_5',
  'templ'

In [47]:
reload(h5)
reload(et)

<module 'file_tools.experiment' from 'file_tools/experiment.py'>